In [ ]:
%run "C:\Repos\AICourse\PersonalFinanceAgent\plugins\email_plugin\email_plugin.ipynb"

## Semantic Kernel

In [ ]:
import os
import asyncio
from dotenv import load_dotenv

# Import Semantic Kernel components
import semantic_kernel as sk
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)
from semantic_kernel.contents import ChatHistory, ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions import KernelArguments

from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent

from semantic_kernel.agents import (
    ChatCompletionAgent,
    ChatHistoryAgentThread,
)
from semantic_kernel.connectors.ai.function_choice_behavior import (
    FunctionChoiceBehavior,
)

from rich.console import Console
console = Console()

# Load environment variables from .env file
load_dotenv("../.env", override=True)

In [ ]:
def create_kernel_with_service(service_id="default"):
    """Create a kernel with either Azure OpenAI or OpenAI service based on available credentials."""
    kernel = sk.Kernel()

    # Try to use Azure OpenAI first
    if (
        os.getenv("AZURE_OPENAI_API_KEY")
        and os.getenv("AZURE_OPENAI_ENDPOINT")
        and os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
    ):
        print("Using Azure OpenAI service")
        kernel.add_service(
            AzureChatCompletion(
                service_id=service_id,
                deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
        )
    # Fall back to OpenAI
    elif os.getenv("OPENAI_API_KEY"):
        print("Using OpenAI service")
        kernel.add_service(
            OpenAIChatCompletion(
                service_id=service_id,
                api_key=os.getenv("OPENAI_API_KEY"),
                ai_model_id="gpt-4",
            )
        )
    else:
        raise ValueError(
            "No AI service credentials found. Please set up your .env file."
        )

    return kernel

In [ ]:
# Create our first kernel with a service ID
email_classifier_service_id = "email_classifier"
email_claissifier_kernel = create_kernel_with_service(service_id=email_classifier_service_id)
print("Kernel created successfully for the email claissifier agent!")

In [ ]:
email_claissifier_kernel.add_plugin(EmailPlugin(), plugin_name="emails")

In [ ]:
# Get the execution settings for our service
email_claissifier_settings = email_claissifier_kernel.get_prompt_execution_settings_from_service_id(service_id=email_classifier_service_id)

# Configure automatic function calling
email_claissifier_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

## Set the service settings for the agent
using_gpt_4o = True

# Example: Adjust the temperature for more creative outputs -- NOTE: RUN THE BELOW ONLY IF YOU ARE USING GPT-4o
if using_gpt_4o:
    email_claissifier_settings.temperature = 0.7
    print(f"Service settings updated: Temperature set to {email_claissifier_settings.temperature}")
else:  # NOTE: RUN THE BELOW ONLY IF YOU ARE USING o1 or o3-mini
    email_claissifier_settings.reasoning_effort = "low"
    print(
        f"Service settings updated: Reasoning Effort set to {email_claissifier_settings.reasoning_effort}"
    )   

In [ ]:
# Create a simple assistant agent
EMAIL_CLASSIFIER_AGENT_NAME = "EmailClassifierAgent"
EMAIL_CLASSIFIER_AGENT_INSTRUCTIONS ="""
You are a email classifier agent that can identify the category of emails as "Finance", "Non-Finance", or "Not classifiable". Ideally you should clearly identify if the emails are finance related or not.

You fetch the emails of the user using the plugin with each email having properties of id, subject, sender, date, and body. Some of these emails contain information about the user's financial transactions and other emails related to the same. For example financial transactions could be in the form of bank transaction alerts or credit card transaction alerts, etc. and other transaction info could be in the form of emails from different providers like Amazon, Flipkart, Netflix, etc. These all emails will help the user later identify the behavior of expenditure based on every area/category they spend in.  You should be able to understand such transactions or related emails as described and filter out the same by tagging them as "Finance", "Non-Finance", or "Not classifiable" emails. Please note that the finance analyst agent will then use this information from you to provide a summary of the user's spend within the month based on categories and sources of payment.

Output: A JSON list of emails having each email with properties of id, subject, sender, date, body, and category of email as classified with key of `email_classification`. The value of `email_classification` should be either "Finance", "Non-Finance", or "Not classifiable".
"""
email_classifier_agent = ChatCompletionAgent(
    kernel=email_claissifier_kernel,
    name=EMAIL_CLASSIFIER_AGENT_NAME,
    instructions=EMAIL_CLASSIFIER_AGENT_INSTRUCTIONS,
)

print(f"Agent '{email_classifier_agent.name}' created successfully!")